In [1]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ec2-user/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ec2-user/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [3]:
import sys
sys.path.append("/home/ec2-user/anaconda3/external/")

# This file contains all the main external libs we'll use
from fastai.imports import *

from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *
import tqdm

In [4]:
PATH = "../Data/kaggle_breast_cancer/"
sz=50
bs=128
arch=resnet34

In [5]:
torch.cuda.is_available()
torch.backends.cudnn.enabled

True

In [6]:
tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
data = ImageClassifierData.from_paths(PATH, tfms=tfms, bs=bs)

In [6]:
# weight = []
# weight.append(len(os.listdir('../Data/kaggle_breast_cancer/train/class_0/')))
# weight.append(len(os.listdir('../Data/kaggle_breast_cancer/train/class_1/')))
# weight_sum = np.sum(np.array(weight))
# weight[0] = weight_sum/weight[0]
# weight[1] = weight_sum/weight[1]
# weight = weight/np.sum(np.array(weight))

# weights = np.ones(len(data.trn_y))*weight[0]
# weights[len(data.trn_y)-np.sum(data.trn_y):] = weight[1]

#weights = torch.DoubleTensor(weights)                                       
#sampler = torch.utils.data.sampler.WeightedRandomSampler(weights,len(weights))                     
#data.trn_dl.sampler = sampler

In [7]:
from fastai.models.cifar10.wideresnet import wrn_22
m = wrn_22()
learn = ConvLearner.from_model_data(m,data)

In [8]:
#keep on the first 11 blocks of the model
m1 = learn.model
l12 = list(m1.children())[0]
l3 = []
for ind, l in enumerate(l12):
    l3.append(l)
    if ind == 11:
        break
m2 = nn.Sequential(*l3)


from fastai.models.cifar10.wideresnet import conv_2d

class tbcnn(nn.Module):
    def __init__(self, m, nout):
        super().__init__()
        self.m = m
        self.nout = nout
        self.cn = conv_2d(384,512,3,2)
        self.ad = nn.AdaptiveMaxPool2d(1)
        self.fl = Flatten()
        self.do = nn.Dropout(0.5)
        self.l1 = nn.Linear(512,nout)
        
        
    def forward(self,inp):
        out1 = self.m(inp)
        out2 = F.relu(self.cn(out1))
        out3 = self.ad(out2)
        out4 = self.fl(out3)
        out5 = self.l1(out4)
        
        return F.log_softmax(out5,dim=-1)
    
new_model = tbcnn(m2,2)

In [9]:
new_model.ad

AdaptiveMaxPool2d(output_size=1)

In [10]:
learn2 = ConvLearner.from_model_data(new_model,data)

In [11]:
learn2.fit(0.01, 10,best_save_name='wrnn_breast_cancer')

epoch      trn_loss   val_loss   accuracy                    
    0      0.368076   0.351532   0.851675  
    1      0.331674   0.376498   0.83545                     
    2      0.319169   0.326342   0.8606                      
    3      0.302549   0.370011   0.845075                    
 20%|█▉        | 183/919 [05:28<22:01,  1.80s/it, loss=0.307]

KeyboardInterrupt: 

In [11]:
learn2.load('wrnn_breast_cancer')

In [12]:
class SaveFeatures():
    features=None
    def __init__(self, m): self.hook = m.register_forward_hook(self.hook_fn)
    def hook_fn(self, module, input, output): self.features = output
    def remove(self): self.hook.remove()

In [13]:
x,y = next(iter(data.val_dl))


In [14]:
sfs = SaveFeatures(new_model.fl)

In [13]:
features = []
outputs = []

#for x,y in iter(data.val_dl):
for x,y in iter(data.val_dl):
    #if len(outputs)>5000:
    #    break
    sfs = SaveFeatures(new_model.fl)
    py = new_model(Variable(x.cuda()))
    sfs.remove()
    features_res = to_np(sfs.features)
    y = to_np(y)
    for i in range(len(y)):
        features.append(np.ravel(features_res[i]))
        outputs.append(y[i])

In [16]:
features = np.array(features)
outputs = np.array(outputs)

In [17]:
features.shape

(40000, 512)

In [18]:
np.save('../Data/kaggle_breast_cancer/val_features.npy',features)
np.save('../Data/kaggle_breast_cancer/val_outputs.npy',outputs)

In [19]:
np.save('../Data/kaggle_breast_cancer/trn_features.npy',features)
np.save('../Data/kaggle_breast_cancer/trn_outputs.npy',outputs)